In [1]:
import Hartmannb


The result of custom_function at [0.95, 0.95, 0.95, 0.95, 0.95, 0.95] is 0.00011397265188861638


In [2]:
import pandas as pd
import torch
from botorch.test_functions.base import BaseTestProblem

import matplotlib.pyplot as plt

In [3]:
from botorch.test_functions.base import BaseTestProblem
from botorch.test_functions import Hartmann
import numpy as np 
import torch 
import random 
from botorch.models import SingleTaskGP
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
# from botorch.fit import fit_gpytorch_model 
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

In [4]:
from scipy import stats
import matplotlib.pyplot as plt
from functools import partial
import torch.nn as nn
from botorch.models import MultiTaskGP
import scipy.stats
from botorch.acquisition import AcquisitionFunction
from botorch.acquisition import LogExpectedImprovement
import pandas as pd

In [5]:
from botorch.acquisition.objective import ScalarizedPosteriorTransform
from botorch.acquisition import qLogNoisyExpectedImprovement
from botorch.acquisition import qLogExpectedImprovement
import os

In [6]:
class NoisyShiftedBranin(BaseTestProblem):
    dim = 2  # Branin 函数是2维的
    _bounds = [(-5, 10), (0, 15)]  # Branin 函数的输入范围

    def __init__(self, noise_std=0.1, shift=None, negate=False):
        super().__init__(negate=negate)
        self.noise_std = noise_std
        self.negate = negate  # 是否取负
        # 如果没有提供shift，就使用0向量
        if shift is not None:
            self.shift = torch.tensor(shift, dtype=torch.float32)
        else:
            self.shift = torch.zeros(self.dim, dtype=torch.float32)

    def evaluate_true(self, X):
        """评估真实的标准Branin函数值"""
        self.shift = self.shift.to(X.device)
        
        # 移位后的输入
        x1 = X[..., 0] + self.shift[0]
        x2 = X[..., 1] + self.shift[1]
        
        # 标准 Branin 函数计算
        term1 = (x2 - 5.1 / (4 * torch.pi**2) * x1**2 + 5 / torch.pi * x1 - 6)**2
        term2 = 10 * (1 - 1 / (8 * torch.pi)) * torch.cos(x1)
        branin_value = term1 + term2 + 10
        
        return branin_value

    def add_noise(self, Y):
        """在评估的Branin函数值上添加噪声"""
        noise = torch.randn_like(Y) * self.noise_std
        return Y + noise

    def forward(self, X):
        """计算带噪声和移位的Branin函数值，并根据需要取负"""
        true_values = self.evaluate_true(X)
        noisy_values = self.add_noise(true_values)

        # 如果需要取负，则返回负值
        if self.negate:
            return -noisy_values
        return noisy_values

In [7]:
ackley_fn1 = NoisyShiftedBranin(noise_std=0, shift=torch.tensor([0, 0]), negate=True)
ackley_fn2 = NoisyShiftedBranin(noise_std=0, shift=torch.tensor([1.0, 1.0]), negate=True)
ackley_fn3 = NoisyShiftedBranin(noise_std=10, shift=torch.tensor([0, 0]), negate=True)

C:\Users\10925\AppData\Local\Temp\ipykernel_1060\3175764114.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [8]:
lower_bounds = [-5, 0.0]  # Lower bounds for each dimension
upper_bounds = [10.0, 15.0]  # Upper bounds for each dimension

In [9]:
initial_points_task11 = [
    Hartmannb.generate_initial_samples_with_custom_function1(
        5, 5, 2, Hartmannb.torch.tensor(lower_bounds), Hartmannb.torch.tensor(upper_bounds), seed=i
    )[0] for i in range(10)
]

initial_points_task22 = None  # If you do not have initial points for task 2, you can set it to None

initial_points_task23 = [
    Hartmannb.generate_initial_samples_with_custom_function1(
        10, 10, 2, Hartmannb.torch.tensor(lower_bounds), Hartmannb.torch.tensor(upper_bounds), seed=i
    )[0] for i in range(20)
]

In [10]:
def fit_multi_task_gp_model_and_select_next_point(train_x, train_obj, device, task):
    input_dim = train_x.shape[1]  # 输入维度（2 维）
    outcome_dim = train_obj.shape[1]  # 输出维度

    num_samples = train_x.size(0)
    task = task.expand(num_samples)  # 扩展 task 张量以匹配 train_x 的样本数

    # 将任务维度拼接到 train_x
    train_x_with_task = torch.cat([train_x, task.unsqueeze(-1)], dim=-1)

    # 修改 Normalize 变换的维度，以匹配拼接后的 train_x 维度
    input_transform = Normalize(d=train_x_with_task.shape[1])

    # 创建多任务模型，区分不同任务的训练数据
    model = MultiTaskGP(
        train_X=train_x_with_task,
        train_Y=train_obj,
        task_feature=train_x_with_task.shape[1] - 1,  # 任务特征索引
        input_transform=input_transform,  # 使用修改后的输入变换
        outcome_transform=Standardize(m=outcome_dim)
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    # 选择下一个采样点
    X_baseline = train_x[train_obj.flatten() == best_value.item()]

    qLogNEI = qLogNoisyExpectedImprovement(model=model, best_f=best_value, X_baseline=X_baseline)
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogNEI,
        bounds=torch.tensor([[-5.0, 0.0], [10.0, 15.0]], device=device, dtype=torch.float64),  # 调整后的边界
        q=1,
        num_restarts=20,
        raw_samples=100,
        options={},
    )

    # 确保 new_point_analytic 是二维张量，形状为 (1, input_dim)
    new_point_analytic = new_point_analytic.squeeze(0)  # 确保形状为 (input_dim,)

    return new_point_analytic

In [11]:
def fit_multi_task_gp_model_and_select_next_point1(train_x, train_obj, device, task):
    input_dim = train_x.shape[1]
    outcome_dim = train_obj.shape[1]
    
    num_samples = train_x.size(0)
    task = task.expand(num_samples)  # 扩展 task 张量以匹配 train_x 的样本数

    # 将任务维度拼接到 train_x
    train_x_with_task = torch.cat([train_x, task.unsqueeze(-1)], dim=-1)

    # 修改 Normalize 变换的维度，以匹配拼接后的 train_x 维度
    input_transform = Normalize(d=train_x_with_task.shape[1])

    # 创建和拟合多任务高斯过程模型
    model = MultiTaskGP(
        train_X=train_x_with_task,  # 使用拼接后的 train_x_with_task
        train_Y=train_obj,
        task_feature=-1,  # 假设最后一个维度是任务特征
        input_transform=input_transform,  # 使用匹配维度的 Normalize
        outcome_transform=Standardize(m=1),  # 假设输出为1维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    # 使用 ScalarizedPosteriorTransform 来标量化多输出模型
    weights = torch.ones(1, device=device)  # 假设输出为1维
    posterior_transform = ScalarizedPosteriorTransform(weights=weights)

    # 使用 qLogExpectedImprovement 选择下一个采样点
    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value, posterior_transform=posterior_transform)


    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)

    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogEI,
        bounds=torch.tensor([[-5.0, 0.0], [10.0, 15.0]], device=device, dtype=torch.float64),  # 调整后的边界
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

In [12]:
results0 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task23,  # 传递生成的初始样本
    fit_task_fn=fit_multi_task_gp_model_and_select_next_point,
    device='cuda',
    task_type='multi'
)

tensor([[ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127],
        [ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127],
        [ 7.8064,  6.0524],
        [ 6.1312, 10.4318],
        [-0.2868,  2.5869],
        [-1.1311, 11.8061],
        [ 5.2864,  5.1166]], device='cuda:0', dtype=torch.float64)


C:\Users\10925\AppData\Roaming\Python\Python311\site-packages\linear_operator\utils\interpolation.py:71: UserWarning:

The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:80.)

C:\Users\10925\AppData\Roaming\Python\Python311\site-packages\linear_operator\utils\interpolation.py:71: UserWarning:

torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:623.)

F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch

Nr. of optimization run: 0
New candidates are: tensor([[2.2664, 2.7606]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 1.12 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



Nr. of optimization run: 1
New candidates are: tensor([[6.7667, 1.2995]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.67 seconds


...

F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



Nr. of optimization run: 3
New candidates are: tensor([[5.0553, 0.9376]], device='cuda:0', dtype=torch.float64)
Iteration 3 took 0.72 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



Nr. of optimization run: 4
New candidates are: tensor([[2.9343, 2.3124]], device='cuda:0', dtype=torch.float64)
Iteration 4 took 0.66 seconds


KeyboardInterrupt: 

In [13]:
def fit_gp_model_and_select_next_point(train_x, train_obj, device):
    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=2),  # 2 维输入
        outcome_transform=Standardize(m=1),
    ).to(device)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    # 使用 LogExpectedImprovement 选择下一个采样点
    X_baseline = train_x[train_obj.flatten() == best_value.item()]

    qLogNEI = qLogNoisyExpectedImprovement(model=model, best_f=best_value, X_baseline=X_baseline)
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogNEI,
        bounds=torch.tensor([[-5.0, 0.0], [10.0, 15.0]], device=device, dtype=torch.float64),  # 调整后的边界
        q=1,
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

In [14]:
def fit_gp_model_and_select_next_point1(train_x, train_obj, device):
    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=train_x.shape[1]),  # 使用输入的维度
        outcome_transform=Standardize(m=1),  # 输出为一维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()



    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value)


    
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogEI,
        bounds=torch.tensor([[-5.0, 0.0], [10.0, 15.0]], device=device, dtype=torch.float64),  # 调整后的边界
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

In [15]:
results1 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

tensor([[ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127]], device='cuda:0', dtype=torch.float64)
Nr. of optimization run: 0
New candidates are: tensor([[6.3795, 0.0000]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 0.67 seconds
Nr. of optimization run: 1
New candidates are: tensor([[5.3639, 2.7239]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.48 seconds
Nr. of optimization run: 2
New candidates are: tensor([[4.8643, 0.0000]], device='cuda:0', dtype=torch.float64)
Iteration 2 took 0.80 seconds
Nr. of optimization run: 3
New candidates are: tensor([[10.,  0.]], device='cuda:0', dtype=torch.float64)
Iteration 3 took 0.64 seconds
Nr. of optimization run: 4
New candidates are: tensor([[10.0000,  1.5258]], device='cuda:0', dtype=torch.float64)
Iteration 4 took 1.24 seconds
Nr. of optimization run: 5
New candidates are: tensor([[10.0000,  2.4389]], device='cuda:0', dtype=torch.float64)
I

In [16]:
def generate_initial_samples(num_samples, dim, lower_bound, upper_bound):
    # 生成指定区间的均匀分布样本
    samples = Hartmannb.np.zeros((num_samples, dim))
    for i in range(dim):
        samples[:, i] = Hartmannb.np.random.uniform(low=lower_bound[i], high=upper_bound[i], size=num_samples)
    return samples

In [17]:
def fit_gp_model_and_select_next_point2(train_x, train_obj, device):
    # 生成样本点，维度为2，区间上下限分别为 [-5.0, 10.0] 和 [0.0, 15.0]
    initial_samples_np = generate_initial_samples(num_samples=1, dim=2, lower_bound=[-5.0, 0.0], upper_bound=[10.0, 15.0])
    
    # 转换为张量并设置数据类型
    initial_samples_tensor = Hartmannb.torch.tensor(initial_samples_np, dtype=Hartmannb.torch.float64)
    
    # 如果生成的数据是3维的张量，去掉第一个维度
    return initial_samples_tensor.squeeze(0)

In [18]:
results2 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point2,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

tensor([[ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127]], device='cuda:0', dtype=torch.float64)
Nr. of optimization run: 0
New candidates are: tensor([[-4.3799, 11.9092]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 0.00 seconds
Nr. of optimization run: 1
New candidates are: tensor([[ 2.8333, 13.4220]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.02 seconds
Nr. of optimization run: 2
New candidates are: tensor([[-4.7004,  1.9904]], device='cuda:0', dtype=torch.float64)
Iteration 2 took 0.00 seconds
Nr. of optimization run: 3
New candidates are: tensor([[4.1164, 9.1499]], device='cuda:0', dtype=torch.float64)
Iteration 3 took 0.00 seconds
Nr. of optimization run: 4
New candidates are: tensor([[8.3524, 8.0103]], device='cuda:0', dtype=torch.float64)
Iteration 4 took 0.00 seconds
Nr. of optimization run: 5
New candidates are: tensor([[ 3.3880, 11.4212]], device='cuda:0', dtype=torch.

In [19]:
def fit_gp_model_and_select_next_point2(train_x, train_obj, device):
        # 生成样本点
    initial_samples_np = Hartmannb.generate_initial_samples(num_samples=1, dim=2, lower_bound=0, upper_bound=1)
    
    # 转换为张量
    initial_samples_tensor = torch.tensor(initial_samples_np, dtype=torch.float64)
    return initial_samples_tensor.squeeze(0)

In [20]:
# 示例使用
num_features = 2
num_samples = 400
Hartmannb.sample_provider = Hartmannb.SampleProvider(num_features, num_samples)



# 检索一个样本（每次调用函数时都会这样做）
next_sample = fit_gp_model_and_select_next_point2(None, None, None)
print(next_sample)

tensor([0.8281, 0.5396], device='cuda:0', dtype=torch.float64)


C:\Users\10925\AppData\Local\Temp\ipykernel_14132\694590683.py:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [21]:
initial_points_task1 = [
    Hartmannb.generate_initial_samples_with_custom_function(
        5 ,0, 2, lower_bounds, upper_bounds, seed=i
    )[0] for i in range(10)
]
initial_points_task2 = None  # 如果你没有任务2的初始点，可以设置为 None

In [22]:
results3 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    fit_task_fn=Hartmannb.zj,  # 你的模型拟合函数
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task1,  # 传递生成的初始样本
    initial_points_task2=initial_points_task2,  # 传递生成的初始样本
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

tensor([[ 3.0530,  5.4109],
        [ 9.0594,  1.5138],
        [-2.6291, 10.3715],
        [-1.9743, 12.9128],
        [ 6.1514,  6.3012]], device='cuda:0')
Nr. of optimization run: 0
New candidates are: tensor([[0.4933, 0.8871]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 0.00 seconds
Nr. of optimization run: 1
New candidates are: tensor([[0.3754, 0.8772]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.00 seconds
Nr. of optimization run: 2
New candidates are: tensor([[0.3398, 0.8058]], device='cuda:0', dtype=torch.float64)
Iteration 2 took 0.00 seconds
Nr. of optimization run: 3
New candidates are: tensor([[0.1177, 0.7268]], device='cuda:0', dtype=torch.float64)
Iteration 3 took 0.02 seconds
Nr. of optimization run: 4
New candidates are: tensor([[0.6865, 0.5387]], device='cuda:0', dtype=torch.float64)
Iteration 4 took 0.00 seconds
Nr. of optimization run: 5
New candidates are: tensor([[0.8472, 0.1700]], device='cuda:0', dtype=torch.float64)
Iteration 5 took 0.0

In [23]:
results4 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task23,  # 传递生成的初始样本
    fit_task_fn=fit_multi_task_gp_model_and_select_next_point1,
    device='cuda',
    task_type='multi'
)

tensor([[ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127],
        [ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127],
        [ 7.8064,  6.0524],
        [ 6.1312, 10.4318],
        [-0.2868,  2.5869],
        [-1.1311, 11.8061],
        [ 5.2864,  5.1166]], device='cuda:0', dtype=torch.float64)
Nr. of optimization run: 0
New candidates are: tensor([[6.6324, 0.0000]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 0.70 seconds
Nr. of optimization run: 1
New candidates are: tensor([[1.9543, 0.0000]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.72 seconds
Nr. of optimization run: 2
New candidates are: tensor([[3.0569, 2.0018]], device='cuda:0', dtype=torch.float64)
Iteration 2 took 0.75 seconds
Nr. of optimization run: 3
New candidates are: tensor([[2.7772, 2.4475]], device='cuda:0', dtype=torch.float64)
Ite

In [24]:
results5 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point1,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

tensor([[ 2.1266,  8.8879],
        [ 3.6815,  0.5568],
        [ 9.2600, 12.9352],
        [-3.1931,  3.9216],
        [-1.7758, 14.6127]], device='cuda:0', dtype=torch.float64)
Nr. of optimization run: 0
New candidates are: tensor([[6.5992, 0.0000]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 0.39 seconds
Nr. of optimization run: 1
New candidates are: tensor([[-5., 15.]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.50 seconds
Nr. of optimization run: 2
New candidates are: tensor([[4.8721, 2.7809]], device='cuda:0', dtype=torch.float64)
Iteration 2 took 0.36 seconds
Nr. of optimization run: 3
New candidates are: tensor([[-5.0000, 11.6065]], device='cuda:0', dtype=torch.float64)
Iteration 3 took 0.53 seconds
Nr. of optimization run: 4
New candidates are: tensor([[4.1340, 0.0000]], device='cuda:0', dtype=torch.float64)
Iteration 4 took 0.67 seconds
Nr. of optimization run: 5
New candidates are: tensor([[10.,  0.]], device='cuda:0', dtype=torch.float64)
Iteration

In [25]:
results_list = [results0,results1, results2, results3, results4, results5]
results_names = ['results0','results1', 'results2', 'results3' ,'results4', 'results5']

# 初始化一个空字典来存储展平后的数据
flattened_results = {}

# 遍历所有结果数组
for i, results in enumerate(results_list):
    if isinstance(results, torch.Tensor):
        results_np = results.cpu().numpy()  # 转为 numpy 数组
    else:
        results_np = results  # 如果已经是 numpy 数组，直接使用
    
    # 获取数组的维度
    shape_len = len(results_np.shape)
    
    if shape_len == 3:
        # 3D 数组的形状 (num_experiments, num_samples, num_metrics)
        num_experiments, num_samples, num_metrics = results_np.shape
        # 展平为 2D 数组
        flattened_array = results_np.reshape(num_experiments * num_samples, num_metrics)
    elif shape_len == 2:
        # 2D 数组的形状 (num_experiments, num_metrics)
        flattened_array = results_np
        num_experiments, num_metrics = flattened_array.shape
    else:
        raise ValueError(f"Unexpected number of dimensions: {shape_len}")
    
    # 将每个数组的列命名为 'results1_Metric_1', 'results1_Metric_2', ...
    columns = [f"{results_names[i]}_Metric_{j+1}" for j in range(num_metrics)]
    
    # 将展平后的数组存入字典，方便后面转换为 DataFrame
    flattened_results.update({col: flattened_array[:, j] for j, col in enumerate(columns)})

# 创建 DataFrame
df_results = pd.DataFrame(flattened_results)

# 文件名
csv_filename = 'all_results_branin.噪声2.csv'

# 检查 CSV 文件是否已经存在
if os.path.exists(csv_filename):
    # 如果文件存在，先读取现有文件
    df_existing = pd.read_csv(csv_filename)
    # 合并新结果到现有的 DataFrame 中
    df_results = pd.concat([df_existing, df_results], ignore_index=True)

# 保存或追加为 CSV 文件
df_results.to_csv(csv_filename, index=False)

print(f"所有结果已保存到 {csv_filename} 文件中。")

所有结果已保存到 all_results_branin.噪声2.csv 文件中。
